# Simularium Conversion Tutorial : MEDYAN Data

In [1]:
from IPython.display import Image
import numpy as np
from simulariumio.medyan import MedyanConverter, MedyanData
from simulariumio import MetaData, DisplayData, ModelMetaData, InputFileData
from simulariumio.filters import TranslateFilter

This notebook provides example python code for converting your own simulation trajectories into the format consumed by the Simularium Viewer. It creates a .simularium JSON file which you can drag and drop onto the viewer like this:

![title](img/drag_drop.gif)

***
## Prepare your spatial data

The Simularium `MedyanConverter` consumes discrete cell data from MEDYAN. 

The converter requires a `MedyanData` object as a parameter ([see documentation](https://allen-cell-animated.github.io/simulariumio/simulariumio.medyan.html#simulariumio.medyan.medyan_data.MedyanData)).

If you'd like to specify PDB or OBJ files or color for rendering an agent type, add a `DisplayData` object for that agent type, as shown below ([see documentation](https://allen-cell-animated.github.io/simulariumio/simulariumio.data_objects.html#module-simulariumio.data_objects.display_data)).


In [2]:
box_size = 1000.0
scale = 0.1

example_data = MedyanData(
    meta_data=MetaData(
        box_size=np.array([box_size, box_size, box_size]),
        scale_factor=scale,
        trajectory_title="Some parameter set",
        model_meta_data=ModelMetaData(
            title="Some agent-based model",
            version="8.1",
            authors="A Modeler",
            description=(
                "An agent-based model run with some parameter set"
            ),
            doi="10.1016/j.bpj.2016.02.002",
            source_code_url="https://github.com/allen-cell-animated/simulariumio",
            source_code_license_url="https://github.com/allen-cell-animated/simulariumio/blob/main/LICENSE",
            input_data_url="https://allencell.org/path/to/native/engine/input/files",
            raw_output_data_url="https://allencell.org/path/to/native/engine/output/files",
        ),
    ),
    snapshot_file=InputFileData(
        file_path="../simulariumio/tests/data/medyan/snapshot.traj",
    ),
    filament_display_data={
        0: DisplayData(
            name="Filament",
            radius=5.0,
            color="#ff1493",
        ),
    },
    linker_display_data={
        0: DisplayData(
            name="LinkerA",
            radius=8.0,
            color="#ffffff",
        ),
        1: DisplayData(
            name="LinkerB",
            radius=8.0,
        ),
        2: DisplayData(
            name="LinkerC",
            radius=8.0,
        ),
    },
    motor_display_data={
        1: DisplayData(
            name="Motor",
            radius=2.0,
            color="#0080ff",
        ),
    },
    agents_with_endpoints=["Motor"]
)

## Convert and save as .simularium JSON file

Once your data is shaped like in the `example_data` object, you can use the converter to generate the file at the given path:

(since MEDYAN coordinates are all positive, use a `TranslateFilter` to center the data in the viewer.)

In [3]:
c = MedyanConverter(example_data)
translation_magnitude = -(box_size * scale) / 2
filtered_data = c.filter_data([
    TranslateFilter(
        translation_per_type={},
        default_translation=translation_magnitude * np.ones(3)
    ),
])
c.write_external_JSON(filtered_data, "example_medyan")

Reading MEDYAN Data -------------
Filtering: translation -------------
Writing JSON (external)-------------
Converting Trajectory Data -------------
saved to example_medyan.simularium


## Visualize in the Simularium viewer

In a supported web-browser (Firefox or Chrome), navigate to https://simularium.allencell.org/ and import your file into the view.